<h1><center>Minimal Covers</center></h1>

Devise and implement three Python functions that compute a minimal cover, all accessible (by the algorithm given in the lecture) minimal covers, and all possible minimal covers of a given list of functional dependencies, respectively. 
Complete the provided Jupyter Notebook using the Python 3 kernel. 
Do not change the name and the parameters of the functions in the template. 
You may add ancillary functions if necessary or convenient.
You may import additional Python standard libraries but no external libraries.

For all questions, use the following data structures.
A functional dependency is represented as a list of two lists. For
example, the functional dependency {A, B} → {C} is represented as [['A', 'B'], ['C']]. 
A set of functional dependencies is represented as a list of functional dependencies. 

Ensure that your functions process the input and produce the output in the format specified in the corresponding questions.

For simplicity, you may assume that the input is always valid (e.g. no empty string, no incomplete functional dependency).
Consider the case in which a functional dependency's left- or right-hand side is empty.

The code should be readable and adequately commented.

The marking team considers clarity of the code and comments, quality of the algorithm and  implementation, and correctness and efficiency of the code.

# Question 1 (4 points)
Write a function *min_cover* that takes a list of functional dependencies as input and returns a minimal cover of the functional dependencies, for example,
````
min_cover([[['A'], ['B', 'C']],[['B'], ['C','D']], [['D'], ['B']], [['A','B','E'], ['F']]])
outputs: [[['A'], ['B']], [['B'], ['C']], [['B'], ['D']], [['D'], ['B']], [['A', 'E'], ['F']]]
````
The results is a minimal cover, i.e. a list of functional dependencies.
There might be more than one correct answer to this question.

In [3]:
import itertools


def remove_empty_lhs_rhs(fds):
    """
    Remove all the empty lhs or rhs case in the functional dependencies
    :param fds:
    :return:
    """
    res = []
    for fd_group in fds:
        left, right = fd_group
        if not left or not right:
            continue
        res.append(fd_group)
    return res


def find_subsets(s):
    """
    Given a set of attributes, return a list of all the proper subsets of the attributes
    input: {'A','B','C'}
    output: [{'A'}, {'A', 'B'}, {'A', 'C'}, {'B'}, {'B', 'C'}, {'C'}]
    :param s: a set of attributes
    :return: a list of sets, each set is a subset of the input set
    """
    s = list(s)
    subsets = []
    for r in range(1, len(s)):
        for subset in itertools.combinations(s, r):
            subsets.append(set(subset))
    return subsets


def find_closure_str_style(attributes, dependencies):
    """
    Given a series of attributes and dependencies, return the closure of the attributes
    :param attributes:
    :param dependencies:
    :return:
    """
    closure = set(attributes)
    omega = []
    # 'A,C->B'
    for item in dependencies:
        left, right = item.split('->')
        left_ls = left.split(',')
        omega.append((set(left_ls), right))
    unused = omega[:]
    # [({'A'},'B'),({'A'},'C')]
    while len(unused) > 0:
        not_changed = True
        for i in range(len(unused)):
            if i >= len(unused):
                break
            dependency = unused[i]
            if dependency[0].issubset(closure):
                closure.add(dependency[1])
                not_changed = False
                unused.pop(i)
        if not_changed:
            break
    return closure


def fds_to_str_style(fds):
    """
    convert a set of functional dependencies to a list of strings
    :param fds: a list of functional dependencies
    :return: a list of strings
    """

    def fd_to_str(a):
        """
        to convert a functional dependency like [['A','B'], ['C']] to a string 'A,B->C'
        :param a:
        :return:
        """
        return ','.join(a[0]) + '->' + ','.join(a[1])

    ans = []
    for fd in fds:
        ans.append(fd_to_str(fd))
    return ans


def find_closure(attributes, dependencies):
    '''
    Given a series of attributes and dependencies, return the closure of the attributes
    :param attributes:
    :param dependencies:
    :return:
    '''
    closure = set(attributes)
    omega = []
    for item in dependencies:
        for right in item[1]:
            omega.append((set(item[0]), right))
    unused = omega[:]
    # [({'A'},'B'),({'A'},'C')]
    while len(unused) > 0:
        not_changed = True
        for i in range(len(unused)):
            if i >= len(unused):
                break
            dependency = unused[i]
            if dependency[0].issubset(closure):
                closure.add(dependency[1])
                not_changed = False
                unused.pop(i)
        if not_changed:
            break
    return closure


def remove_duplicates(lst):
    """
    remove duplicates in a list of lists
    :param lst:
    :return:
    """
    return [list(x) for x in set(tuple(sorted(x)) for x in lst)]


def handle_right_minimal_Q1(fds):
    """
    simplify the rhs of functional dependencies for Q1
    :return:
    """
    first_step_gen = []
    for fd in fds:
        left, right = fd.split('->')
        lhs_set = set(left.split(','))
        if len(right) > 1:
            for item in right.split(','):
                # if item not in lhs_set:
                first_step_gen.append(left + '->' + item)
        else:
            first_step_gen.append(fd)
    return first_step_gen


def simplify_lhs_Q1(fds):
    """
    simplify the lhs of the functional dependencies,
    if the subsets of the lhs implies the rhs, then replace the lhs by one of the valid subsets
    return all possible combinations of the replacements
    :param fds:
    :return:
    """

    def check_subset_closure(sub_sub, right, closure_memo):
        key = tuple(sorted(list(sub_sub)))
        closure = closure_memo[key] or find_closure_str_style(sub_sub, fds)
        closure_memo[key] = closure
        return right in closure

    closure_memo = dict()
    substitution = []

    for fd in fds:
        flag_replacement_fd = False
        left, right = fd.split('->')
        fd_lhs_subsets = find_subsets(left.split(','))
        for subset in fd_lhs_subsets:
            key = tuple(sorted(list(subset)))
            # indicates that the subset is possibly a replacement of the lhs of the fd
            closure_subset = closure_memo.get(key, None) or find_closure_str_style(subset, fds)
            closure_memo[key] = closure_subset
            if right in closure_memo[key]:
                if len(subset) > 1:
                    flag = True
                    # still need to check if the subset is a valid replacement
                    sub_sub_set = find_subsets(subset)
                    for sub_sub in sub_sub_set:
                        check_subset = check_subset_closure(sub_sub, right, closure_memo)
                        # for any subset of the subset SS is valid, then this subset SS is invalid
                        if check_subset:
                            flag = False
                            break
                    # print(subset, right)
                    # else the subset is valid and can be a replacement
                    if flag:
                        replacement_fd = ','.join(sorted(list(subset))) + '->' + right
                        substitution.append(replacement_fd)
                        flag_replacement_fd = True
                        break
                else:
                    # the subset only has one element, it is a valid replacement
                    replacement_fd = ','.join(sorted(list(subset))) + '->' + right
                    substitution.append(replacement_fd)
                    flag_replacement_fd = True
                    break
        if not flag_replacement_fd:
            substitution.append(fd)
    return substitution


def handle_whole_minimal_Q1(sec_step_gen):
    """
    handle the whole set simplification especially for Q1
    need to simplify the set itself by removing functional dependencies that can be derived
    :param sec_step_gen
    :return: the simplified set
    """

    def remove_redundant_fd(fds):
        """
        Remove all redundant fds
        check if rhs is still
        entailed in the absence of fd, if true then remove it
        :param fds:
        :return:
        """
        for fd in fds:
            fd_without_one = fds[:]
            fd_without_one.remove(fd)
            left, right = fd.split('->')
            if right in left:
                fds.remove(fd)
            elif right in find_closure_str_style(left, fd_without_one):
                fds.remove(fd)
        return fds

    return remove_redundant_fd(sec_step_gen)


def str_to_list_style_Q1(fds):
    """
    convert a list of strings to a list of lists
    :param fds:
    :return:
    """
    ans = []
    for fd in fds:
        left, right = fd.split('->')
        ans.append([left.split(','), [right]])
    return ans


def min_cover(fds):
    """
    find all the reachable minimal covers of fds
    :param fds:
    :return:
    """

    # remove all the empty lhs or rhs case in the functional dependencies
    fds = remove_empty_lhs_rhs(fds)
    # convert the fds to string style
    fds = fds_to_str_style(fds)
    # simplify the rhs of the fds
    fds = handle_right_minimal_Q1(fds)
    # simplify the lhs of the fds
    fds = simplify_lhs_Q1(fds)

    fds = handle_whole_minimal_Q1(fds)

    return str_to_list_style_Q1(fds)


### Expected Result

In [4]:
fds = [[['A'], ['B', 'C']],[['B'], ['C','D']], [['D'], ['B']], [['A','B','E'], ['F']]]
fds =  [[['A', 'B', 'C'], ['B', 'D', 'E', 'A', 'C', 'G', 'H']], [['B'], ['D', 'E', 'B']], [['G'], ['B', 'D', 'E']]]

min_cover(fds)

[[['A'], ['A']],
 [['A', 'B', 'C'], ['G']],
 [['A', 'B', 'C'], ['H']],
 [['B'], ['D']],
 [['B'], ['E']],
 [['G'], ['B']],
 [['G'], ['E']]]

**The result you get should be similar to:**
```
[[['A'], ['B']],
 [['B'], ['C']],
 [['B'], ['D']],
 [['D'], ['B']],
 [['A', 'E'], ['F']]]
```

# Question 2 (3 points) 
Write a function *min_covers* that takes a list of functional dependencies as input and returns all minimal covers reachable from the given list of functional dependencies, for example,
````
min_cover([[['A'], ['A', 'B']], [['B'], ['A','C']], [['A'], ['C']], [['A','B'], ['C']]])
outputs: [
            [[['A'], ['B']], [['B'], ['A']], [['B'], ['C']]],
            [[['A'], ['B']], [['B'], ['A']], [['A'], ['C']]]
        ]
````
The results is a list of minimal covers.
Make sure that you clearly explain the rationale of your solution in comments of the code.

In [21]:
import itertools


def remove_empty_lhs_rhs(fds):
    """
    Remove all the empty lhs or rhs case in the functional dependencies
    :param fds:
    :return:
    """
    res = []
    for fd_group in fds:
        left, right = fd_group
        if not left or not right:
            continue
        res.append(fd_group)
    return res


def find_subsets(s):
    """
    Given a set of attributes, return a list of all the proper subsets of the attributes
    input: {'A','B','C'}
    output: [{'A'}, {'A', 'B'}, {'A', 'C'}, {'B'}, {'B', 'C'}, {'C'}]
    :param s: a set of attributes
    :return: a list of sets, each set is a subset of the input set
    """
    s = list(s)
    subsets = []
    for r in range(1, len(s)):
        for subset in itertools.combinations(s, r):
            subsets.append(set(subset))
    return subsets


def fd_to_str(a):
    """
    to convert a functional dependency like [['A','B'], ['C']] to a string 'A,B->C'
    :param a:
    :return:
    """
    return ','.join(a[0]) + '->' + ','.join(a[1])


def fds_to_str_style(fds):
    """
    convert a set of functional dependencies to a list of strings
    :param fds: a list of functional dependencies
    :return: a list of strings
    """
    ans = []
    for fd in fds:
        ans.append(fd_to_str(fd))
    return ans

def find_closure(attributes, dependencies):
    '''
    Given a series of attributes and dependencies, return the closure of the attributes
    :param attributes:
    :param dependencies:
    :return:
    '''
    closure = set(attributes)
    omega = []
    for item in dependencies:
        for right in item[1]:
            omega.append((set(item[0]), right))
    unused = omega[:]
    # [({'A'},'B'),({'A'},'C')]
    while len(unused) > 0:
        not_changed = True
        for i in range(len(unused)):
            if i >= len(unused):
                break
            dependency = unused[i]
            if dependency[0].issubset(closure):
                closure.add(dependency[1])
                not_changed = False
                unused.pop(i)
        if not_changed:
            break
    return closure

def remove_duplicates(lst):
    """
    remove duplicates in a list of lists
    :param lst:
    :return:
    """
    return [list(x) for x in set(tuple(sorted(x)) for x in lst)]
def find_all_subsets_closure(fds, attributes, closure_cache):
    """
    find the closure of all the subsets of the attributes
    :param fds:
    :param attributes:
    :return:
    """
    subsets = find_subsets(attributes)
    for s in subsets:
        closure_cache[tuple(sorted(list(s)))] = find_closure(s, fds)
    return closure_cache

def handle_right_minimal_Q2(fds):
    """
    simplify the rhs of functional dependencies for Q2
    :param: fds
    :return:
    """
    first_step_gen = []
    for fd in fds:
        left, right = fd.split('->')
        lhs_set = set(left.split(','))
        if len(right) > 1:
            for item in right.split(','):
                # if item not in lhs_set:
                first_step_gen.append(left + '->' + item)
        else:
            first_step_gen.append(fd)
    return first_step_gen


def simplify_lhs(fds, closure_cache, lhs_replacement_cache):
    """
    simplify the lhs of the functional dependencies,
    if the subsets of the lhs implies the rhs, then replace the lhs by one of the valid subsets
    return all possible combinations of the replacements
    :param fds:
    :return:
    """
    def check_subset_closure(sub_sub, right):
        return right in closure_cache[tuple(sorted(list(sub_sub)))]

    def generate_combinations(index, current_combination):
        # Base case: if we've gone through all FDs, add the current combination to the result
        if index == len(fds):
            return [current_combination]

        # Get the current FD
        fd = fds[index]

        # If the FD can be replaced, generate combinations with each replacement
        if fd in lhs_replacement_cache:
            all_combinations = []
            for replacement in lhs_replacement_cache[fd]:
                with_replacement = generate_combinations(index + 1, current_combination + [replacement])
                all_combinations.extend(with_replacement)
            return all_combinations
        else:
            # If the FD can't be replaced, just add it to the current combination and continue
            return generate_combinations(index + 1, current_combination + [fd])

    for fd in fds:
        left, right = fd.split('->')
        fd_lhs_subsets = find_subsets(left.split(','))
        for subset in fd_lhs_subsets:
            # indicates that the subset is possibly a replacement of the lhs of the fd
            if right in closure_cache[tuple(sorted(list(subset)))]:
                if len(subset) > 1:
                    flag = True
                    # still need to check if the subset is a valid replacement
                    sub_sub_set = find_subsets(subset)
                    for sub_sub in sub_sub_set:
                        check_subset = check_subset_closure(sub_sub, right)
                        # for any subset of the subset SS is valid, then this subset SS is invalid
                        if check_subset:
                            flag = False
                            break
                    # print(subset, right)
                    # else the subset is valid and can be a replacement
                    if flag:
                        replacement_fd = ','.join(sorted(list(subset))) + '->' + right
                        lhs_replacement_cache[fd] = lhs_replacement_cache.get(fd,[])
                        lhs_replacement_cache[fd].append(replacement_fd)
                else:
                    # the subset only has one element, it is a valid replacement
                    replacement_fd = ','.join(sorted(list(subset))) + '->' + right
                    lhs_replacement_cache[fd] = lhs_replacement_cache.get(fd,[])
                    lhs_replacement_cache[fd].append(replacement_fd)

    return generate_combinations(0, [])

def handle_whole_minimal_Q2(sec_step_gen):
    """
    handle the whole set simplification especially for Q2
    need to simplify the set itself by removing functional dependencies that can be derived
    :param sec_step_gen
    :return: the simplified set
    """

    def find_closure_str_style(attributes, dependencies):
        """
        Given a series of attributes and dependencies, return the closure of the attributes
        :param attributes:
        :param dependencies:
        :return:
        """
        closure = set(attributes)
        omega = []
        # 'A,C->B'
        for item in dependencies:
            left, right = item.split('->')
            left_ls = left.split(',')
            omega.append((set(left_ls), right))
        unused = omega[:]
        # [({'A'},'B'),({'A'},'C')]
        while len(unused) > 0:
            not_changed = True
            for i in range(len(unused)):
                if i >= len(unused):
                    break
                dependency = unused[i]
                if dependency[0].issubset(closure):
                    closure.add(dependency[1])
                    not_changed = False
                    unused.pop(i)
            if not_changed:
                break
        return closure

    def not_redundant(p):
        for i in range(len(p)):
            _left, _right = p[i].split('->')
            fd_without_one = p[:i] + p[i + 1:]
            if _right in find_closure_str_style(_left.split(','), fd_without_one):
                return False
        return True

    def dfs(fds, index, path, result):
        if index == len(fds) and not_redundant(path):
            result.append(path[:])
            return
        elif index == len(fds):
            return
        this_fd = fds[index]
        right = this_fd.split('->')[1]
        fds_without_this = fds[:index] + fds[index + 1:]
        attributes = this_fd.split('->')[0].split(',')
        closure = find_closure_str_style(attributes, fds_without_this)
        # if this fd can be entailed by others
        if right in closure:
            # Case 1: Remove the current FD and do recursion
            dfs(fds_without_this, index, path, result)
            # Case 2: Keep the current FD and do recursion (backtracking)
            dfs(fds, index + 1, path + [this_fd], result)
        else:
            # If the current FD cannot be entailed by other FDs, keep it and move to the next FD
            dfs(fds, index + 1, path + [this_fd], result)

    def find_all_combinations(fds):
        result = []
        dfs(fds, 0, [], result)
        return result

    return find_all_combinations(sec_step_gen)


def find_all_attributes(fds):
    attributes = set()
    for fd in fds:
        for attr_list in fd:
            for attr in attr_list:
                attributes.add(attr)
    return list(attributes)

def str_to_list_style(fdss):
    """
    convert a list of a list of strings to a 4D list
    :param fdss:
    :return:
    """
    ans = []
    for fds in fdss:
        fds_ls_style = []
        for fd in fds:
            left, right = fd.split('->')
            fds_ls_style.append([left.split(','), [right]])
        ans.append(fds_ls_style)
    return ans

def min_covers(fds):
    """
    find all the reachable minimal covers of fds
    :param fds:
    :return:
    """

    # closure_cache is a dictionary that stores the closure of all the subsets of the attributes
    closure_cache = dict()
    # lhs_replacement_cache is a dictionary that stores the possible replacements of the lhs of the fds
    lhs_replacement_cache = dict()
    # find all the attributes
    attributes = find_all_attributes(fds)
    # remove all the empty lhs or rhs case in the functional dependencies
    fds = remove_empty_lhs_rhs(fds)
    # find the closure of all the subsets of the attributes(this step need to be executed first
    find_all_subsets_closure(fds, attributes, closure_cache)
    # convert the fds to string style
    fds = fds_to_str_style(fds)
    # simplify the rhs of the fds
    fds = handle_right_minimal_Q2(fds)
    # simplify the lhs of the fds
    fds = simplify_lhs(fds, closure_cache, lhs_replacement_cache)

    final_result = []

    for sec_step_gen in fds:
        # simplify the whole set and add the simplified result to the whole result
        final_step_gen = handle_whole_minimal_Q2(sec_step_gen)
        for dependencies in final_step_gen:
            final_result.append(dependencies)
    # remove all the duplicates in the final result
    fdss = remove_duplicates(final_result)
    return str_to_list_style(fdss)


### Expected Result

In [22]:
fds = [[['A'], ['A', 'B']], [['B'], ['A','C']], [['A'], ['C']], [['A','B'], ['C']]]
min_covers(fds)

[[[['A'], ['B']], [['B'], ['A']], [['B'], ['C']]],
 [[['A'], ['B']], [['A'], ['C']], [['B'], ['A']]]]

**The result you get should be similar to:**
```
[
    [[['A'], ['B']], [['B'], ['A']], [['B'], ['C']]],
    [[['A'], ['B']], [['B'], ['A']], [['A'], ['C']]]
]
```

# Question 3 (3 points)
* Write a function *all_min_covers* that takes a list of functional dependencies as input and returns all minimal covers, for example,
````
all_min_covers([[['A'], ['B']],[['B'], ['C']], [['C'], ['A']]])
outputs: [
	[[['A'], ['C']], [['B'], ['A']], [['C'], ['A']], [['A'], ['B']]],
	[[['B'], ['C']], [['C'], ['A']], [['A'], ['B']]],
	[[['C'], ['B']], [['A'], ['C']], [['C'], ['A']], [['B'], ['C']]],
	[[['C'], ['B']], [['A'], ['C']], [['B'], ['A']]],
	[[['B'], ['C']], [['A'], ['B']], [['B'], ['A']], [['C'], ['B']]]
]
````

The results is a list of minimal covers.
Make sure that you clearly explain the rationale of your solution in comments of the code.
This question is challenging and not everyone may find a solution.

In [25]:
import itertools


def remove_empty_lhs_rhs_Q3(fds):
    """
    Remove all the empty lhs or rhs case in the functional dependencies
    :param fds:
    :return:
    """
    res = []
    for fd_group in fds:
        left, right = fd_group
        if not left or not right:
            continue
        res.append(fd_group)
    return res


def find_subsets_Q3(s):
    """
    Given a set of attributes, return a list of all the proper subsets of the attributes
    input: {'A','B','C'}
    output: [{'A'}, {'A', 'B'}, {'A', 'C'}, {'B'}, {'B', 'C'}, {'C'}]
    :param s: a set of attributes
    :return: a list of sets, each set is a subset of the input set
    """
    s = list(s)
    subsets = []
    for r in range(1, len(s)):
        for subset in itertools.combinations(s, r):
            subsets.append(set(subset))
    return subsets



def fds_to_str_style(fds):
    """
    convert a set of functional dependencies to a list of strings
    :param fds: a list of functional dependencies
    :return: a list of strings
    """

    def fd_to_str(a):
        """
        to convert a functional dependency like [['A','B'], ['C']] to a string 'A,B->C'
        :param a:
        :return:
        """
        return ','.join(a[0]) + '->' + ','.join(a[1])

    ans = []
    for fd in fds:
        ans.append(fd_to_str(fd))
    return ans


def find_closure(attributes, dependencies):
    """
    Given a series of attributes and dependencies, return the closure of the attributes
    :param attributes:
    :param dependencies:
    :return:
    """
    closure = set(attributes)
    omega = []
    for item in dependencies:
        for right in item[1]:
            omega.append((set(item[0]), right))
    unused = omega[:]
    # [({'A'},'B'),({'A'},'C')]
    while len(unused) > 0:
        not_changed = True
        for i in range(len(unused)):
            if i >= len(unused):
                break
            dependency = unused[i]
            if dependency[0].issubset(closure):
                closure.add(dependency[1])
                not_changed = False
                unused.pop(i)
        if not_changed:
            break
    return closure


def remove_duplicates(lst):
    """
    remove duplicates in a list of lists
    :param lst:
    :return:
    """
    return [list(x) for x in set(tuple(sorted(x)) for x in lst)]


def find_all_subsets_closure(fds, attributes, closure_cache):
    """
    find the closure of all the subsets of the attributes
    :param fds:
    :param attributes:
    :return:
    """
    subsets = find_subsets_Q3(attributes)
    for s in subsets:
        closure_cache[tuple(sorted(list(s)))] = find_closure(s, fds)
    return closure_cache


def handle_right_minimal_Q3(fds):
    """
    simplify the rhs of functional dependencies
    :return:
    """
    first_step_gen = []
    for fd in fds:
        left, right = fd.split('->')
        lhs_set = set(left.split(','))
        if len(right) > 1:
            for item in right.split(','):
                # if item not in lhs_set:
                first_step_gen.append(left + '->' + item)
        else:
            first_step_gen.append(fd)
    return first_step_gen


def simplify_lhs(fds, closure_cache, lhs_replacement_cache):
    """
    simplify the lhs of the functional dependencies,
    if the subsets of the lhs implies the rhs, then replace the lhs by one of the valid subsets
    return all possible combinations of the replacements
    :param fds:
    :return:
    """

    def check_subset_closure(sub_sub, right):
        return right in closure_cache[tuple(sorted(list(sub_sub)))]

    def generate_combinations(index, current_combination):
        # Base case: if we've gone through all FDs, add the current combination to the result
        if index == len(fds):
            return [current_combination]

        # Get the current FD
        fd = fds[index]

        # If the FD can be replaced, generate combinations with each replacement
        if fd in lhs_replacement_cache:
            all_combinations = []
            for replacement in lhs_replacement_cache[fd]:
                with_replacement = generate_combinations(index + 1, current_combination + [replacement])
                all_combinations.extend(with_replacement)
            return all_combinations
        else:
            # If the FD can't be replaced, just add it to the current combination and continue
            return generate_combinations(index + 1, current_combination + [fd])

    for fd in fds:
        left, right = fd.split('->')
        fd_lhs_subsets = find_subsets_Q3(left.split(','))
        for subset in fd_lhs_subsets:
            # indicates that the subset is possibly a replacement of the lhs of the fd
            if right in closure_cache[tuple(sorted(list(subset)))]:
                if len(subset) > 1:
                    flag = True
                    # still need to check if the subset is a valid replacement
                    sub_sub_set = find_subsets_Q3(subset)
                    for sub_sub in sub_sub_set:
                        check_subset = check_subset_closure(sub_sub, right)
                        # for any subset of the subset SS is valid, then this subset SS is invalid
                        if check_subset:
                            flag = False
                            break
                    # print(subset, right)
                    # else the subset is valid and can be a replacement
                    if flag:
                        replacement_fd = ','.join(sorted(list(subset))) + '->' + right
                        lhs_replacement_cache[fd] = lhs_replacement_cache.get(fd, [])
                        lhs_replacement_cache[fd].append(replacement_fd)
                else:
                    # the subset only has one element, it is a valid replacement
                    replacement_fd = ','.join(sorted(list(subset))) + '->' + right
                    lhs_replacement_cache[fd] = lhs_replacement_cache.get(fd, [])
                    lhs_replacement_cache[fd].append(replacement_fd)

    return generate_combinations(0, [])


def handle_whole_minimal_Q3(sec_step_gen):
    """
    handle the whole set simplification especially for Q3
    need to simplify the set itself by removing functional dependencies that can be derived
    :param sec_step_gen
    :return: the simplified set
    """

    def find_closure_str_style(attributes, dependencies):
        """
        Given a series of attributes and dependencies, return the closure of the attributes
        :param attributes:
        :param dependencies:
        :return:
        """
        closure = set(attributes)
        omega = []
        # 'A,C->B'
        for item in dependencies:
            left, right = item.split('->')
            left_ls = left.split(',')
            omega.append((set(left_ls), right))
        unused = omega[:]
        # [({'A'},'B'),({'A'},'C')]
        while len(unused) > 0:
            not_changed = True
            for i in range(len(unused)):
                if i >= len(unused):
                    break
                dependency = unused[i]
                if dependency[0].issubset(closure):
                    closure.add(dependency[1])
                    not_changed = False
                    unused.pop(i)
            if not_changed:
                break
        return closure

    def not_redundant(p):
        for i in range(len(p)):
            _left, _right = p[i].split('->')
            fd_without_one = p[:i] + p[i + 1:]
            if _right in find_closure_str_style(_left.split(','), fd_without_one):
                return False
        return True

    def dfs(fds, index, path, result):
        if index == len(fds) and not_redundant(path):
            result.append(path[:])
            return
        elif index == len(fds):
            return
        this_fd = fds[index]
        right = this_fd.split('->')[1]
        fds_without_this = fds[:index] + fds[index + 1:]
        attributes = this_fd.split('->')[0].split(',')
        closure = find_closure_str_style(attributes, fds_without_this)
        # if this fd can be entailed by others
        if right in closure:
            # Case 1: Remove the current FD and do recursion
            dfs(fds_without_this, index, path, result)
            # Case 2: Keep the current FD and do recursion (backtracking)
            dfs(fds, index + 1, path + [this_fd], result)
        else:
            # If the current FD cannot be entailed by other FDs, keep it and move to the next FD
            dfs(fds, index + 1, path + [this_fd], result)

    def find_all_combinations(fds):
        result = []
        dfs(fds, 0, [], result)
        return result

    return find_all_combinations(sec_step_gen)


def find_all_attributes(fds):
    attributes = set()
    for fd in fds:
        for attr_list in fd:
            for attr in attr_list:
                attributes.add(attr)
    return list(attributes)


def str_to_list_style(fdss):
    """
    convert a list of strings to a list of lists
    :param fdss:
    :return:
    """
    ans = []
    for fds in fdss:
        fds_ls_style = []
        for fd in fds:
            left, right = fd.split('->')
            fds_ls_style.append([left.split(','), [right]])
        ans.append(fds_ls_style)
    return ans


def generate_all_reachable(fds, closure_cache):
    """
    Generate all the reachable functional dependencies
    :param fds:
    :return:
    """
    ans = []
    for fd in fds:
        left, right = fd
        closure = closure_cache[tuple(sorted(left))]
        for i in closure:
            if i != left[0]:
                ans.append([left, [i]])
    return ans


def all_minimal_covers(fds):
    # closure_cache is a dictionary that stores the closure of all the subsets of the attributes
    closure_cache = dict()
    # lhs_replacement_cache is a dictionary that stores the possible replacements of the lhs of the fds
    lhs_replacement_cache = dict()

    # find all the attributes
    attributes = find_all_attributes(fds)
    # remove all the empty lhs or rhs case in the functional dependencies
    fds = remove_empty_lhs_rhs_Q3(fds)
    # find the closure of all the subsets of the attributes(this step need to be executed first
    find_all_subsets_closure(fds, attributes, closure_cache)
    # Firstly generate all the reachable functional dependencies
    fds = generate_all_reachable(fds, closure_cache)
    # convert the fds to string style
    fds = fds_to_str_style(fds)
    # simplify the rhs of the fds
    fds = handle_right_minimal_Q3(fds)
    # simplify the lhs of the fds
    fds = simplify_lhs(fds, closure_cache, lhs_replacement_cache)

    final_result = []

    for sec_step_gen in fds:
        # simplify the whole set and add the simplified result to the whole result
        final_step_gen = handle_whole_minimal_Q3(sec_step_gen)
        for dependencies in final_step_gen:
            final_result.append(dependencies)
    # remove all the duplicates in the final result
    fdss = remove_duplicates(final_result)
    return str_to_list_style(fdss)


### Expected Result

In [26]:
fds = [[['A'], ['B']],[['B'], ['C']], [['C'], ['A']]]
all_minimal_covers(fds)

[[[['A'], ['B']], [['B'], ['C']], [['C'], ['A']]],
 [[['A'], ['C']], [['B'], ['C']], [['C'], ['A']], [['C'], ['B']]],
 [[['A'], ['B']], [['B'], ['A']], [['B'], ['C']], [['C'], ['B']]],
 [[['A'], ['B']], [['A'], ['C']], [['B'], ['A']], [['C'], ['A']]],
 [[['A'], ['C']], [['B'], ['A']], [['C'], ['B']]]]

**The result you get should be:**
```
[
	[[['A'], ['C']], [['B'], ['A']], [['C'], ['A']], [['A'], ['B']]],
	[[['B'], ['C']], [['C'], ['A']], [['A'], ['B']]],
	[[['C'], ['B']], [['A'], ['C']], [['C'], ['A']], [['B'], ['C']]],
	[[['C'], ['B']], [['A'], ['C']], [['B'], ['A']]],
	[[['B'], ['C']], [['A'], ['B']], [['B'], ['A']], [['C'], ['B']]]
]
````